# Maxwell's Daemon
### Final Project for Computer Based Physical Modelling
### Thomas de Paula Barbosa - 3749592

# Theory
## About the Daemon
Maxwell's daemon is a thought experiment made to test the validity the Second Law of Thermodynamics. The thought experiment consists in two isolated boxes filled with gas which have a gate in between them. The gate's opening and closing is controlled by a daemon who only allows molecules that have a certain velocity to pass through the gate. After a set amount of time, this means that faster molecules will be on one side, while the slower molecules will be in the other side. This would potentially violate the 2nd Law due to the entropy of the system decreasing

## The 2nd Law of Thermodynamics
The 2nd Law of Thermodynamics states that in an isolated system, entropy can only increase.


## Brief Note on Code
While Python is dynamically typed, I like giving type hints for keeping track of things more easily.

In [1]:
from ipycanvas import hold_canvas, Canvas, MultiCanvas
import numpy as np


CONTAINER_COORDINATES = np.array([800, 600])  # x, y
# the daemon will open the gate for a particle when it reaches this speed
SPEED_THRESHHOLD: float = 10.0
GATE_SIZE: int = 100
X, Y = 0, 1  # this is just for ease of reading
PARTICLE_RADIUS: int = 5


class Particle:
    def __init__(self, position: np.ndarray, velocity: np.ndarray):
        self.position: np.ndarray = position
        self.velocity: np.ndarray = velocity
        self._speed: float = np.hypot(velocity[0], velocity[1])
        self.isHot: bool = True if self._speed >= SPEED_THRESHHOLD else False

    
    def __repr__(self):
        return f"Particle({self.position}, {self.velocity}, {self._speed}, {self.isHot})"

    
    def _bounce(self, position, boundary, coordinate):
        if position <= PARTICLE_RADIUS:
            position = 2*(PARTICLE_RADIUS) - position
            self.velocity[coordinate] *= -1 
        elif position >= boundary - PARTICLE_RADIUS:
            position = 2*(boundary - PARTICLE_RADIUS) - position
            self.velocity[coordinate] *= -1
        
        return position
    
    
    def _isAtGate(self, upper_boundary, gate_size, point) -> bool:
        if 0.5*(upper_boundary - gate_size) < point < upper_boundary - 0.5*(upper_boundary + gate_size) + gate_size:
            return True
        else:
            return False
        
    def _isOnLeft(self, container, point) -> bool:
        if 0 <= point[X] <= container[X]/2:
            return True
        else:
            return False
    
    def update(self):
        new_position = self.position + self.velocity
        
        # right boundary breach
        if new_position[X] + PARTICLE_RADIUS >= CONTAINER_COORDINATES[X]: 
            new_position[X] = self._bounce(new_position[X], CONTAINER_COORDINATES[X], X)
        #left boundary breach
        elif new_position[X] - PARTICLE_RADIUS <= 0: 
            new_position[X] = self._bounce(new_position[X], CONTAINER_COORDINATES[X], X)
            
        # upper boundary breach
        if new_position[Y] + PARTICLE_RADIUS >= CONTAINER_COORDINATES[Y]: 
            new_position[Y] = self._bounce(new_position[Y], CONTAINER_COORDINATES[Y], Y)
        # lower boundary breach
        elif new_position[Y] - PARTICLE_RADIUS <= 0:   
            new_position[Y] = self._bounce(new_position[Y], CONTAINER_COORDINATES[Y], Y)
            
            
        # TODO REMOVE NOT
        if not self.isHot:
            # heading right to left
            if self.velocity[X] < 0 and not self._isOnLeft(CONTAINER_COORDINATES, new_position):
                if self._isAtGate(CONTAINER_COORDINATES[Y], GATE_SIZE, new_position[Y]):
                    # we can pass through!
                    pass
                elif self.velocity < 0:
                    # we're not at the gate. Behave as if hitting a wall
                    new_position[X] = self._bounce(new_position[X], CONTAINER_COORDINATES[X]/2, X)
            # heading left to right
            elif self.velocity[X] >= 0 and not self._isOnLeft(CONTAINER_COORDINATES, new_position):
                # will hit the rightmost wall
                if new_position[X] + PARTICLE_RADIUS >= CONTAINER_COORDINATES[X]:
                    new_position[X] = self._bounce(new_position[X], CONTAINER_COORDINATES[X], X)
            elif self.velocity[X] >= 0 and self._isOnLeft(CONTAINER_COORDINATES, new_position):
                # will hit middle
                new_position[X] = self._bounce(new_position[X], CONTAINER_COORDINATES[X]/2, X)
           
        # this was copied
        #if self.isHot:
        #    if 0 <= new_position[X] < CONTAINER_COORDINATES[X]/2 + PARTICLE_RADIUS:
        #       if self.velocity[X] > 0 and 0 < new_position[Y] < CONTAINER_COORDINATES[Y] - GATE_SIZE/2 and new_position[X] >= CONTAINER_COORDINATES[X] - PARTICLE_RADIUS:
        #           self._bounce(new_position[X], CONTAINER_COORDINATES[X]/2, X)
        #       elif self.velocity[X] > 0 and CONTAINER_COORDINATES[Y]/2 + GATE_SIZE/2 < new_position[Y] < CONTAINER_COORDINATES[Y] and new_position[X] >= CONTAINER_COORDINATES[X]/2 - PARTICLE_RADIUS:
        #           self._bounce(new_position[X], CONTAINER_COORDINATES[X]/2, X)
        #   
        #   if self.velocity[X] < 0 and new_position[X] > CONTAINER_COORDINATES[X]/2:
        #       if new_position[X] < CONTAINER_COORDINATES[X]/2 + PARTICLE_RADIUS:
        #           self._bounce(new_position[X], CONTAINER_COORDINATES[X]/2, X)
        
        #ew_position[X] = self._bounce(new_position[X], CONTAINER_COORDINATES[X], X)
        #ew_position[Y] = self._bounce(new_position[Y], CONTAINER_COORDINATES[Y], Y)
        self.position = new_position
            

## Creating the Canvas

In [1]:
from ipycanvas import hold_canvas, Canvas, MultiCanvas
import numpy as np


CONTAINER_COORDINATES = np.array([800, 600])  # x, y
# the daemon will open the gate for a particle when it reaches this speed
SPEED_THRESHHOLD: float = 10.0
GATE_SIZE: int = 100
X, Y = 0, 1  # this is just for ease of reading
PARTICLE_RADIUS: int = 5

In [2]:
%load_ext autoreload
%autoreload complete
from particle_class import *
molecules = []
N_PARTICLES = 50
for _ in range(N_PARTICLES):
    molecules.append(
        Particle(
            position=np.array([
                np.random.randint(PARTICLE_RADIUS, CONTAINER_COORDINATES[X] - PARTICLE_RADIUS + 1),
                np.random.randint(PARTICLE_RADIUS, CONTAINER_COORDINATES[Y] - PARTICLE_RADIUS + 1),
            ]),
            velocity=np.random.random(2) * np.random.choice([-1, 1], size=2) * 10,
            radius=PARTICLE_RADIUS
        )
    )

# force a hot molecule to be on wrong side
molecules[0].position = np.array([CONTAINER_COORDINATES[X]/2 + 20, CONTAINER_COORDINATES[Y]/2 - 200])
# molecules[0].velocity = np.array([20, 20])
# assert np.hypot(molecules[0].velocity[0], molecules[0].velocity[1]) >= SPEED_THRESHHOLD, "speed too low"
# force a cold molecule to be on the wrong size
molecules[1].position = np.array([CONTAINER_COORDINATES[X]/2 - 20, CONTAINER_COORDINATES[Y]/2 - 200])
# molecules[1].velocity = np.array([-10, 10])
# assert np.hypot(molecules[0].velocity[0], molecules[0].velocity[1]) >= SPEED_THRESHHOLD, "speed too high"

In [3]:
# NOTE ipycanvas seems to have a bug where numbers must be converted to python's int
# this happens despite the fact that the documentation claims it should be compatible with numpy
cc = Canvas(width=CONTAINER_COORDINATES[X], height=CONTAINER_COORDINATES[Y])
container_middle = int(CONTAINER_COORDINATES[X]/2), int(CONTAINER_COORDINATES[Y]/2)
STEPS = 2_000
with hold_canvas(cc):
    for _ in range(STEPS):
        cc.clear()
        
        # box
        cc.stroke_style = "black"
        cc.stroke_rect(1, 1, int(CONTAINER_COORDINATES[X]-1), int(CONTAINER_COORDINATES[Y]-1))
        # middle line
        cc.stroke_line(
            container_middle[X], 0, 
            container_middle[X], int(container_middle[Y] - GATE_SIZE/2)
        )
        cc.stroke_line(
            container_middle[X], int(CONTAINER_COORDINATES[Y]),
            container_middle[X], int(container_middle[Y] + GATE_SIZE/2)
        )
        # gate/daemon
        cc.stroke_style = "violet"
        cc.stroke_line(
            container_middle[X], int(container_middle[Y] - GATE_SIZE/2),
            container_middle[X], int(container_middle[Y] + GATE_SIZE/2)
        )
        
        for m in molecules:
            m.update()
            if m.isHot:
                cc.fill_style = "red"
            else:
                cc.fill_style = "blue"
            # draw molecule
            cc.fill_circle(int(m.position[X]), int(m.position[Y]), PARTICLE_RADIUS)
        cc.sleep(20)
display(cc)

Canvas(height=600, width=800)

In [5]:
good_side = 0
escaped = 0
for m in molecules:
    if m._isOnCorrectSide():
        good_side += 1
    if m.position[X] > CONTAINER_COORDINATES[X] or m.position[Y] > CONTAINER_COORDINATES[Y]:
        escaped += 1
        #raise RuntimeError("A molecule breached containment!")
    elif m.position[X] < 0 or m.position[Y] < 0:
        escaped += 1
        #raise RuntimeError("A molecule breached containment!")
incorrect_molecules = len(molecules) - good_side
percentage = escaped/len(molecules) * 100
print(f"Good side: {good_side}\nBad side: {incorrect_molecules}")
print(f"{escaped} molecule(s) escaped. That's {percentage:.2}%")


Good side: 27
Bad side: 23
1 molecule(s) escaped. That's 2.0%
